## Zbieranie danych

In [1]:
import glob
import pandas as pd
from io import StringIO

# Path where the TSV files are stored
path_to_tsv_files = './datasets/*.tsv'

# List to hold DataFrames with the specified column
dataframes_with_column = []

# The column of interest
column_of_interest = '2018'

# Function to preprocess the file content
def preprocess_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        # Read the file and replace tabs with commas
        content = file.read().replace('\t', ',')
    return content

# Loop through each TSV file in the directory
for tsv_file in glob.glob(path_to_tsv_files):
    try:
        # Preprocess the file content
        file_content = preprocess_file(tsv_file)

        # Use StringIO to simulate a file object for pandas
        df = pd.read_csv(StringIO(file_content))

        # Strip spaces from column names
        df.columns = df.columns.str.strip()

        # Check if both the specified column and 'geo\\TIME_PERIOD' are in the DataFrame
        if column_of_interest in df.columns and 'geo\\TIME_PERIOD' in df.columns:
            # Keep only the '2018' column and 'geo\\TIME_PERIOD'
            df = df[['geo\\TIME_PERIOD', column_of_interest]]
            dataframes_with_column.append(df)
    except Exception as e:
        print(f"Error processing file {tsv_file}: {e}")


In [2]:
dataframes_with_column.pop(4)

for i, df in enumerate(dataframes_with_column):
    df['geo\\TIME_PERIOD'] = df['geo\\TIME_PERIOD'].astype('category')
    if df['2018'].dtype == 'object':
        df['2018'] = pd.to_numeric(df['2018'], errors='coerce')
    print('len of', i, len(df))

len of 0 869
len of 1 37
len of 2 126
len of 3 2329
len of 4 23955
len of 5 17646
len of 6 3990
len of 7 2244
len of 8 95640
len of 9 21713
len of 10 3471
len of 11 605767
len of 12 744
len of 13 26928


In [ ]:
import pandas as pd

# Assuming dataframes_with_column is already filled with DataFrames
if dataframes_with_column:
    # Rename columns outside of the merge loop to prevent issues with index lookup
    for i, df in enumerate(dataframes_with_column):
        df.rename(columns={column_of_interest: f'col{i+1}'}, inplace=True)

    # Start with the first DataFrame
    merged_df = dataframes_with_column[0]

    # Merge remaining DataFrames
    for i in range(1, len(dataframes_with_column)):
        merged_df = pd.merge(merged_df, dataframes_with_column[i], on='geo\\TIME_PERIOD', how='outer')
        print(f"Merged {i+1} of {len(dataframes_with_column)} DataFrames.")

    print(merged_df.head())  # Display the first few rows of the merged DataFrame
else:
    print("No dataframes contained the required columns.")

Merged 2 of 14 DataFrames.
Merged 3 of 14 DataFrames.
Merged 4 of 14 DataFrames.
Merged 5 of 14 DataFrames.


In [ ]:
print(merged_df)

In [3]:
import glob
import pandas as pd
from io import StringIO

# Path where the TSV files are stored
path_to_tsv_files = './datasets/*.tsv'

# List to hold DataFrames with the specified column
dataframes_with_column = []

# The column of interest
column_of_interest = '2018'

# Function to preprocess the file content
def preprocess_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        # Read the file and replace tabs with commas
        content = file.read().replace('\t', ',')
    return content

# Loop through each TSV file in the directory
for tsv_file in glob.glob(path_to_tsv_files):
    try:
        # Preprocess the file content
        file_content = preprocess_file(tsv_file)

        # Use StringIO to simulate a file object for pandas
        df = pd.read_csv(StringIO(file_content))

        # Strip spaces from column names
        df.columns = df.columns.str.strip()

        # Check if both the specified column and 'geo\\TIME_PERIOD' are in the DataFrame
        if column_of_interest in df.columns and 'geo\\TIME_PERIOD' in df.columns:
            # Keep only the '2018' column and 'geo\\TIME_PERIOD'
            df = df[['geo\\TIME_PERIOD', column_of_interest]]
            dataframes_with_column.append(df)
    except Exception as e:
        print(f"Error processing file {tsv_file}: {e}")

# Concatenate all DataFrames with a unique identifier
if dataframes_with_column:
    for i, df in enumerate(dataframes_with_column):
        df['source'] = f'col{i+1}'  # Add a unique identifier for each source DataFrame

    # Concatenate all DataFrames vertically
    combined_df = pd.concat(dataframes_with_column)

    # Pivot the DataFrame to wide format
    pivoted_df = combined_df.pivot_table(index='geo\\TIME_PERIOD', columns='source', values='2018', aggfunc='first').reset_index()

    print(pivoted_df.head())  # Display the first few rows of the pivoted DataFrame
else:
    print("No dataframes contained the required columns.")


source geo\TIME_PERIOD    col1 col10   col11   col12 col13  col14  col15  \
0                   AL     NaN  0.6      NaN   3830    NaN  34.9   30.1    
1                   AT     : u  0.0   13.0 u  37690    : u   7.9    8.6    
2                   BA      :    NaN      :       :     :     NaN    NaN   
3                   BE  48.1 u  0.3    21.2   35510     :   -3.1   15.9    
4                   BG     : u  0.0      : u   6330    : u  12.2   11.1    

source  col2    col3   col4   col5   col6    col7   col8   col9  
0        NaN  23.46   62.0    207     NaN   1972     NaN   1.1   
1       9.0   22.36   20.9   3157   24.8   24675   66.7    9.6   
2         :      NaN    NaN    NaN     :      NaN     :     NaN  
3       8.4   16.39   23.3   6141    7.7   23135   89.4   11.4   
4       7.7    19.2   62.4    440     : u   3474   95.0   21.7   


In [4]:
print(pivoted_df)

source geo\TIME_PERIOD    col1  col10   col11    col12  col13   col14   col15  \
0                   AL     NaN   0.6      NaN    3830     NaN   34.9    30.1    
1                   AT     : u   0.0   13.0 u   37690     : u    7.9     8.6    
2                   BA      :     NaN      :        :      :      NaN     NaN   
3                   BE  48.1 u   0.3    21.2    35510      :    -3.1    15.9    
4                   BG     : u   0.0      : u    6330     : u   12.2    11.1    
5                   CH  45.0 u   0.1     7.6    61690     : u    9.4     8.0    
6                   CY      :    0.4      : u   24500      :   42.2 u   27.9    
7                   CZ     : u   0.0   16.5 u   17990      :   14.6 d    6.6    
8                   DE   39.3    0.1     7.3    35650      :     7.1    11.6    
9                   DK     : u   0.6    11.3    48450      :    17.4    12.4    
10                  EA     NaN  0.1 e     NaN   30910     NaN     NaN  12.8 e   
11                EA12     N